In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# Load an image for prediction (for example, from a file)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
import cv2

import numpy as np
import mediapipe as mp

In [2]:

img_height, img_width = 64, 64
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255)

train_path = '../Datasets/Eyes_Data/TrainingSet'
test_path = '../Datasets/Eyes_Data/TestSet'
cv_path = '../Datasets/Eyes_Data/CVSet'


train_datagen  = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    shear_range=0.2,  # Random shear transformation
    zoom_range=0.2,  # Random zoom transformation
    horizontal_flip=True  # Random horizontal flip
)


cv_datagen  = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    shear_range=0.2,  # Random shear transformation
    zoom_range=0.2,  # Random zoom transformation
    horizontal_flip=True  # Random horizontal flip
)

train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=batch_size
)

val_gen = cv_datagen.flow_from_directory(
    cv_path,  # Using the new validation set
    target_size=(img_height, img_width),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=batch_size
)

test_gen = datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=batch_size,
    shuffle=False  # So you can match predictions to filenames
)


Found 4103 images belonging to 2 classes.
Found 1704 images belonging to 2 classes.
Found 4232 images belonging to 2 classes.


In [3]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 1)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])



model.summary()

c:\Users\Anas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 682,753 (2.60 MB)

 Trainable params: 682,753 (2.60 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [6]:
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,  # <- CV set used here
    callbacks=[],  # (Optional) Add EarlyStopping or ReduceLROnPlateau if needed
)


c:\Users\Anas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 124s 952ms/step - accuracy: 0.5497 - loss: 0.6844 - val_accuracy: 0.8034 - val_loss: 0.4996
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 13s 99ms/step - accuracy: 0.8511 - loss: 0.3838 - val_accuracy: 0.8369 - val_loss: 0.3987
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - accuracy: 0.8876 - loss: 0.2989 - val_accuracy: 0.8586 - val_loss: 0.3569
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - accuracy: 0.8976 - loss: 0.2550 - val_accuracy: 0.8985 - val_loss: 0.2642
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 13s 97ms/step - accuracy: 0.9132 - loss: 0.2409 - val_accuracy: 0.9108 - val_loss: 0.2477
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 17s 134ms/step - accuracy: 0.9139 - loss: 0.2198 - val_accuracy: 0.8996 - val_loss: 0.2612
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 40s 309ms/step - accuracy: 0.9203 - loss: 0.2136 - val_accuracy: 0.8914 - val_loss: 0.2800
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 13s 97ms/step - accuracy: 0.9228 - loss: 0.205

In [7]:
loss, accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {accuracy:.2f}")


133/133 ━━━━━━━━━━━━━━━━━━━━ 32s 238ms/step - accuracy: 0.7426 - loss: 0.5867
Test Accuracy: 0.84


In [8]:
img_path = ["../Datasets/cc.jpg" , "../Datasets/oo.jpg" , "../Datasets/oo2.jpg"]  # Replace with the image you want to classify


for path in img_path:
    print(f"Processing: {path}")
    
    # Load image in grayscale
    img = load_img(path, color_mode="grayscale", target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
    
    # Reshape to (1, 64, 64, 1)
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    
    if prediction < 0.5:
        print(f"Prediction: {prediction[0][0]:.4f} — 👁️ Closed Eyes")
    else:
        print(f"Prediction: {prediction[0][0]:.4f} — 👁️ Open Eyes")

Processing: ../Datasets/cc.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Prediction: 0.4459 — 👁️ Closed Eyes
Processing: ../Datasets/oo.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Prediction: 0.8774 — 👁️ Open Eyes
Processing: ../Datasets/oo2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: 0.0003 — 👁️ Closed Eyes


In [ ]:
#model.save("../models/open_close_eyes_model.h5")

In [9]:


# === Load your trained model ===
#model = load_model("../models/open_close_eyes_model.h5")  # Replace with actual path
img_size = (64, 64)  # Model input size

# === Load Haar cascades ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

# === Read image ===
img = cv2.imread("../Datasets/d3.jpg")  # Replace with your image path
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# === Detect faces ===
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

for (x, y, w, h) in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    # === Detect eyes in face ROI ===
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex, ey, ew, eh) in eyes:
        # Eye region must be in upper half of face
        if ey > h // 2:
            continue

        # Optional: filter eyes based on size
        if ew < 10 or eh < 10 or ew > w // 2:
            continue
        eye_gray = roi_gray[ey:ey+eh, ex:ex+ew]
        eye_resized = cv2.resize(eye_gray, img_size)  # Resize to 64x64
        eye_array = eye_resized.astype("float32") / 255.0  # Normalize
        eye_array = np.expand_dims(eye_array, axis=-1)  # Add channel dim
        eye_array = np.expand_dims(eye_array, axis=0)   # Add batch dim

        # === Predict ===
        prediction = model.predict(eye_array)[0][0]
        label = "Open" if prediction >= 0.5 else "Closed"

        # === Draw results ===
        cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
        cv2.putText(roi_color, label, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (255, 255, 255), 2)

# === Display final result ===
cv2.imshow("Eye Detection & Classification", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


In [11]:

img_height, img_width = 64, 64
batch_size = 32

# One datagen for both training and validation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Training generator (80%)
train_gen = datagen.flow_from_directory(
    '../Datasets/New_Eyes/data/train',
    target_size=(img_height, img_width),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=batch_size,
    subset='training',
    shuffle=True,
    seed=42
)

# Validation generator (20%)
val_gen = datagen.flow_from_directory(
    '../Datasets/New_Eyes/data/train',
    target_size=(img_height, img_width),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=batch_size,
    subset='validation',
    shuffle=True,
    seed=42
)

# Test generator (separate test set)
test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_directory(
    '../Datasets/New_Eyes/data/test',
    target_size=(img_height, img_width),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=batch_size,
    shuffle=False
)


Found 65340 images belonging to 2 classes.
Found 16335 images belonging to 2 classes.
Found 3223 images belonging to 2 classes.


In [13]:
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,  # <- CV set used here
    callbacks=[],  # (Optional) Add EarlyStopping or ReduceLROnPlateau if needed
)

Epoch 1/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 145s 71ms/step - accuracy: 0.9442 - loss: 0.1593 - val_accuracy: 0.8877 - val_loss: 0.2892
Epoch 2/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 77s 38ms/step - accuracy: 0.9586 - loss: 0.1148 - val_accuracy: 0.8986 - val_loss: 0.2597
Epoch 3/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 77s 37ms/step - accuracy: 0.9673 - loss: 0.0929 - val_accuracy: 0.8984 - val_loss: 0.2671
Epoch 4/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 77s 38ms/step - accuracy: 0.9717 - loss: 0.0828 - val_accuracy: 0.9109 - val_loss: 0.2317
Epoch 5/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 77s 38ms/step - accuracy: 0.9759 - loss: 0.0709 - val_accuracy: 0.8919 - val_loss: 0.2679
Epoch 6/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 77s 38ms/step - accuracy: 0.9763 - loss: 0.0678 - val_accuracy: 0.9055 - val_loss: 0.2379
Epoch 7/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 77s 38ms/step - accuracy: 0.9774 - loss: 0.0632 - val_accuracy: 0.9194 - val_loss: 0.2100
Epoch 8/10
2042/2042 ━━━━━━━━━━━━━━━━━━━━ 78s 38ms/step - accuracy: 0.9793 

In [14]:
loss, accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {accuracy:.2f}")

101/101 ━━━━━━━━━━━━━━━━━━━━ 19s 194ms/step - accuracy: 0.9587 - loss: 0.1393
Test Accuracy: 0.89


In [ ]:
model.save("models/open_close_eyes_model2.h5")

In [ ]:

# === Load your trained model ===
# model = load_model("models/open_close_eyes_model2.h5")  # Replace with actual path

img_size = (64, 64)  # Model input size

# === Load Haar cascades ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

# === Read image ===
img = cv2.imread("../Datasets/o1.jpg")  # Replace with your image path

# === Resize image if too large ===
max_width = 800
max_height = 600
height, width = img.shape[:2]
if width > max_width or height > max_height:
    scale = min(max_width / width, max_height / height)
    img = cv2.resize(img, (int(width * scale), int(height * scale)))

# === Convert to grayscale ===
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# === Detect faces ===
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

for (x, y, w, h) in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    # === Detect eyes in face ROI ===
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex, ey, ew, eh) in eyes:
        # Eye region must be in upper half of face
        if ey > h // 2:
            continue

        # Optional: filter eyes based on size
        if ew < 10 or eh < 10 or ew > w // 2:
            continue

        eye_gray = roi_gray[ey:ey+eh, ex:ex+ew]
        eye_resized = cv2.resize(eye_gray, img_size)  # Resize to 64x64
        eye_array = eye_resized.astype("float32") / 255.0  # Normalize
        eye_array = np.expand_dims(eye_array, axis=-1)  # Add channel dim
        eye_array = np.expand_dims(eye_array, axis=0)   # Add batch dim

        # === Predict ===
        prediction = model.predict(eye_array)[0][0]
        label = "Open" if prediction >= 0.5 else "Closed"

        # === Draw results ===
        cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
        cv2.putText(roi_color, label, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (255, 255, 255), 2)

# === Display final result ===
cv2.imshow("Eye Detection & Classification", img)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [3]:

# === Load trained model ===
model = load_model("models/open_close_eyes_model2.h5")  # Replace with your model path
img_size = (64, 64)

# === Initialize MediaPipe face mesh ===
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

# === Load Haar cascades ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

# === Read image ===
img = cv2.imread("../Datasets/n3.jpg")  # Replace with your image path
if img is None:
    raise ValueError("Image not found!")

# === Resize image if too large ===
max_width, max_height = 800, 600
h, w = img.shape[:2]
if w > max_width or h > max_height:
    scale = min(max_width / w, max_height / h)
    img = cv2.resize(img, (int(w * scale), int(h * scale)))
    h, w = img.shape[:2]

# === Convert BGR to RGB for MediaPipe ===
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = face_mesh.process(rgb_img)

# === Flag to check if any eye was processed ===
eye_detected = False

# === Try MediaPipe ===
if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        left_eye_indices = [33, 133]   # Outer corners of left eye
        right_eye_indices = [362, 263] # Outer corners of right eye

        def extract_eye_region(indices):
            x_coords = [int(face_landmarks.landmark[i].x * w) for i in indices]
            y_coords = [int(face_landmarks.landmark[i].y * h) for i in indices]
            x_min, x_max = min(x_coords), max(x_coords)
            y_min, y_max = min(y_coords), max(y_coords)
            margin_x = int((x_max - x_min) * 0.4)
            margin_y = int((y_max - y_min) * 1.2)
            x1 = max(x_min - margin_x, 0)
            y1 = max(y_min - margin_y, 0)
            x2 = min(x_max + margin_x, w)
            y2 = min(y_max + margin_y, h)
            return x1, y1, x2, y2

        for eye_label, indices in zip(["Left", "Right"], [left_eye_indices, right_eye_indices]):
            x1, y1, x2, y2 = extract_eye_region(indices)
            eye_roi = img[y1:y2, x1:x2]
            if eye_roi.size == 0:
                continue

            eye_gray = cv2.cvtColor(eye_roi, cv2.COLOR_BGR2GRAY)
            eye_resized = cv2.resize(eye_gray, img_size)
            eye_array = eye_resized.astype("float32") / 255.0
            eye_array = np.expand_dims(eye_array, axis=[0, -1])

            prediction = model.predict(eye_array)[0][0]
            label = "Open" if prediction >= 0.5 else "Closed"

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f" {label}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            eye_detected = True

# === If no eyes detected by MediaPipe, fallback to Haar ===
if not eye_detected:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)

        for (ex, ey, ew, eh) in eyes:
            if ey > h // 2 or ew < 10 or eh < 10 or ew > w // 2:
                continue

            eye_gray = roi_gray[ey:ey+eh, ex:ex+ew]
            eye_resized = cv2.resize(eye_gray, img_size)
            eye_array = eye_resized.astype("float32") / 255.0
            eye_array = np.expand_dims(eye_array, axis=[0, -1])

            prediction = model.predict(eye_array)[0][0]
            label = "Open" if prediction >= 0.5 else "Closed"

            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            cv2.putText(roi_color, label, (ex, ey - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

# === Show final result ===
cv2.imshow("Eye Detection & Classification (Fallback Ready)", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
